In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from collections import Counter, OrderedDict
import IPython
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
# CSS = """
# .output {
#     flex-direction: row;
# }
# """
# HTML('<style>{}</style>'.format(CSS))
# pd.set_option('display.max_columns', None)

pd.set_option('display.max_columns', 50)

In [2]:
main_df = pd.read_csv('./data/0826export.csv')
main_df.columns = main_df.columns.str.lower()

store_master_df = pd.read_excel('./documentation/store_master.xlsx')
store_master_df.columns = store_master_df.columns.str.lower()

---

## Mapping Function:

#### To map from `/store_master.xlsx` onto `main_df`

In [3]:
def map_my_dataframe(main_df, store_master_df, a_list, map_on='store') -> 'pandas.DataFrame':
    """
    > a_list = df2 column names' values(for mapping to df1)
    > rids of outlier comp_pct
    > converts descriptive tidy columns to dtype('category')
    """
    
    for category in a_list:
        df_2_dict = dict(zip(store_master_df[map_on], store_master_df[category]))
        main_df[category] = main_df[map_on].map(df_2_dict)
        
    main_df['store_size'] = pd.cut(main_df.gross_feet, bins=[0, 1000, 2000, 3000, 4000], labels=['sml', 'mdm', 'lrg', 'xlrg'])
    main_df.drop(main_df.comp_pct.values.argmax(), inplace=True) # Large outlier, drop fiscal_week with comp_pct > 200.
    main_df.drop(main_df.comp_pct.values.argmax(), inplace=True) # Large outlierr, drop fiscal_week with comp_pct > 20
    main_df.drop('store_zip', axis=1, inplace=True)
    main_df.iloc[:, -1] = main_df.iloc[:, -1].astype('category') # convert last 4 tidy columns to dtype('category')
    
    return main_df  

df = map_my_dataframe(main_df, store_master_df, store_master_df.columns[2:], map_on='store')
cols = [i for i in df.columns if 'ly' in i or 'avg' in i] # dropping all last year
df = df.drop(cols, axis=1)
df.to_pickle('./data/0826export_mapped.pkl')

---

# Difference comparison Function:

#### Function compares percentage differences between top 15%, bot 15% of fiscal monthly observations aggregated by mean.

In [7]:
def top_bot_var_compare(df, category, rows) -> "DataFrame[['diff%', 'top', 'bot'], ['diff%', 'top', 'bot'],[...]].sort_values()":
    """
    !!!DOCUMENTATION HERE!!!
    """
    levels_list = [category, 
                   'store', 
                   'fiscal_year', 
                   'fiscal_month'
                  ]
    cat_list = sorted(df[category].unique())
    
    df = df.groupby(levels_list).mean()
    dict_df = {}
    
    for cat in cat_list:
        cat_df = df.loc[[cat]].iloc[:, 2:].droplevel(levels_list[1:]).drop('strak_comp_traffic_delta', axis=1)        
        sample_percent = int(.15 * len(cat_df))
        
        top_series = round(cat_df.nlargest(sample_percent, 'comp_pct').mean(), 2).rename('top_avg')
        bot_series = round(cat_df.nsmallest(sample_percent, 'comp_pct').mean(), 2).rename('bot_avg')
        diff_series = round(abs(abs(top_series - bot_series) / ((top_series + bot_series) / 2) * 100)).rename('%diff')
        

        together = pd.concat([diff_series, top_series, bot_series], axis=1) 
        together = together.sort_values('%diff', ascending=False)
        together.index.names = ['Sorted Descending']
        together = together.iloc[:, :].reset_index()
        
        dict_df[cat] = together.iloc[1:rows+1, :]
    
    
    return pd.concat(dict_df.values(), axis=1, keys=dict_df.keys())

---

## By Class:
#### `'Mall'`, `'Open Air'`, `'Downtown'`, `'Street'`, `'Airport'`, `'Lifestyle'`, `'Outlet'`

In [8]:
compare_class = top_bot_var_compare(df, 'class', rows=10)
compare_class

Airport                          \
           Sorted Descending %diff  top_avg  bot_avg   
1         trans_cnt_exchg_in  76.0     2.17     0.97   
2          trans_cnt_returns  74.0     4.61     2.13   
3             am_tenure_days  61.0  1858.63   994.64   
4          special_order_amt  57.0  4009.52  2223.92   
5       cp_total_email_trans  55.0   519.98   914.24   
6                multi_value  54.0  4113.66  2374.77   
7             cp_total_trans  53.0   585.92  1013.42   
8   cp_valid_mailing_address  50.0   433.38   718.88   
9            cp_valid_emails  50.0   304.17   505.23   
10               multi_units  49.0    28.66    17.36   

                      Downtown                           \
             Sorted Descending  %diff  top_avg  bot_avg   
1                     ft_count  101.0     0.64     0.21   
2        accessory_other_units   95.0     2.01     0.72   
3                  multi_value   89.0  4534.77  1733.67   
4   accessory_shoe_trees_units   88.0     5.68     2.20   
5   accessory_shoe_trees_value   87.0   140.52    55.27   
6           trans_cnt_exchg_in   86.0     2.77     1.11   
7                  multi_units   86.0    29.97    11.95   
8            special_order_amt   83.0  3641.30  1505.75   
9        accessory_coats_value   82.0  1341.85   560.61   
10       accessory_other_value   81.0    68.77    29.26   

                   Lifestyle                          \
           Sorted Descending %diff  top_avg  bot_avg   
1             ft_tenure_days  65.0  1268.40   648.77   
2             cp_total_trans  57.0  4263.86  7630.33   
3            cp_valid_emails  57.0  2141.82  3830.97   
4       cp_total_email_trans  56.0  3589.97  6377.87   
5   cp_valid_mailing_address  55.0  3567.37  6282.05   
6     trans_cnt_empl_returns  33.0     0.07     0.05   
7            tot_tenure_days  32.0  7693.68  5568.99   
8             am_tenure_days  30.0  1776.45  1315.02   
9          trans_cnt_returns  28.0     5.53     7.34   
10           mgr_tenure_days  27.0  4047.95  3085.48   

                          Mall                          \
             Sorted Descending %diff  top_avg  bot_avg   
1         cp_total_email_trans  47.0   590.31   954.87   
2               cp_total_trans  47.0   694.13  1115.55   
3     cp_valid_mailing_address  46.0   580.99   931.21   
4              cp_valid_emails  46.0   379.51   607.69   
5               ft_tenure_days  39.0   481.18   716.80   
6                  multi_value  30.0  5264.68  3885.48   
7    accessory_shoe_care_value  30.0   367.62   272.00   
8                  multi_units  29.0    35.18    26.24   
9   accessory_shoe_trees_units  28.0     5.21     3.95   
10   accessory_shoe_care_units  27.0    59.79    45.51   

                         Open Air                           \
                Sorted Descending  %diff  top_avg  bot_avg   
1                  ft_tenure_days  103.0   987.15   317.06   
2                        ft_count   63.0     0.52     0.27   
3                  am_tenure_days   56.0   284.71   508.38   
4             trans_cnt_exchg_out   56.0     0.41     0.23   
5   accessory_shirt_sweater_value   54.0  2164.88  1241.92   
6                        am_count   50.0     0.63     0.38   
7                     multi_value   48.0  4142.17  2532.30   
8   accessory_shirt_sweater_units   47.0    32.39    20.07   
9           accessory_other_value   46.0    30.19    48.34   
10                    multi_units   43.0    27.85    17.96   

                        Outlet                          \
             Sorted Descending %diff  top_avg  bot_avg   
1               ft_tenure_days  38.0   347.57   508.27   
2           trans_cnt_exchg_in  35.0     3.49     2.44   
3          trans_cnt_exchg_out  33.0     0.39     0.28   
4    accessory_shoe_care_value  32.0   259.77   188.45   
5        accessory_coats_units  32.0     8.16     5.89   
6             mark_down_amt_ty  31.0  3329.55  2432.40   
7                  multi_units  31.0    68.55    50.40   
8 

---

## By Volume Band:
#### `1`, `2`, `3`, `4`, `5`, `6`

In [6]:
compare_vband = top_bot_var_compare(df, 'volume_band', rows=10)
compare_vband

1                          \
             Sorted Descending %diff  top_avg  bot_avg   
1           trans_cnt_exchg_in  52.0     1.96     1.15   
2   accessory_shoe_trees_units  37.0     2.65     1.82   
3        accessory_coats_units  36.0     5.67     3.95   
4        accessory_coats_value  35.0   805.70   563.91   
5                  multi_units  34.0    31.05    21.94   
6             mark_down_amt_ty  34.0  1612.20  1139.87   
7          trans_cnt_exchg_out  34.0     0.24     0.17   
8   accessory_shoe_trees_value  33.0    57.60    41.38   
9                  multi_value  33.0  3199.25  2297.36   
10           special_order_amt  31.0  1630.37  1192.89   

                            2                          \
            Sorted Descending %diff  top_avg  bot_avg   
1              ft_tenure_days  90.0   147.92   390.95   
2                    ft_count  42.0     0.32     0.49   
3                 multi_units  29.0    41.96    31.23   
4              am_tenure_days  28.0   504.52   671.38   
5                 multi_value  28.0  4638.40  3506.18   
6          trans_cnt_exchg_in  26.0     3.40     2.63   
7   accessory_shoe_care_value  20.0   277.18   226.36   
8            mark_down_amt_ty  20.0  1843.25  1511.44   
9                 shoes_units  20.0    85.85    70.51   
10     trans_cnt_empl_returns  20.0     0.09     0.11   

                           3                          \
           Sorted Descending %diff  top_avg  bot_avg   
1       cp_total_email_trans  64.0   584.41  1130.29   
2   cp_valid_mailing_address  64.0   558.76  1083.24   
3             cp_total_trans  64.0   696.13  1355.04   
4            cp_valid_emails  62.0   358.04   678.89   
5      accessory_other_units  40.0     2.71     1.81   
6             ft_tenure_days  36.0   557.46   804.27   
7      accessory_coats_units  29.0     8.39     6.28   
8                multi_units  28.0    53.68    40.58   
9           mark_down_amt_ty  27.0  2509.34  1914.15   
10         special_order_amt  26.0  3068.97  2368.87   

                             4                          \
             Sorted Descending %diff  top_avg  bot_avg   
1   accessory_shoe_trees_value  36.0   123.98    86.45   
2   accessory_shoe_trees_units  31.0     5.36     3.93   
3               am_tenure_days  31.0  1413.74  1926.71   
4              mgr_tenure_days  30.0  2182.61  2949.50   
5           trans_cnt_exchg_in  30.0     3.67     2.70   
6            trans_cnt_returns  29.0     7.77     5.81   
7            special_order_amt  29.0  3373.53  2528.18   
8                     ft_count  29.0     0.94     1.26   
9               ft_tenure_days  28.0  1448.70  1923.88   
10    trans_cnt_exchg_original  26.0    12.95     9.93   

                            5                           \
            Sorted Descending  %diff  top_avg  bot_avg   
1             cp_valid_emails  111.0   222.56   772.28   
2        cp_total_email_trans  110.0   361.06  1237.60   
3              cp_total_trans  109.0   432.22  1466.15   
4    cp_valid_mailing_address  109.0   357.03  1208.11   
5              am_tenure_days   60.0  2168.45  4033.85   
6       accessory_other_value   45.0   126.80   200.62   
7       accessory_other_units   35.0     3.22     4.60   
8   accessory_shoe_care_value   28.0   483.38   365.30   
9   accessory_shoe_care_units   27.0    83.83    63.57   
10          special_order_amt   25.0  4973.80  3859.67   

                                6                            
                Sorted Descending  %diff   top_avg  bot_avg  
1        cp_valid_mailing_address  175.0    252.35  3710.17  
2                  cp_total_trans  172.0    346.28  4533.25  
3                 cp_valid_emails  171.0    181.23  2283.76  
4            cp_total_email_trans  170.0    310.11  3801.18  
5      accessory_shoe_trees_units  100.0      2.18     6.53  
6      accessory_shoe_trees_value  100.0     55.23   166.26  
7   accessory_shirt_sweater_value   50.0  11256.16  6735.30  
8   accessory_shirt_sw

---

## By RPT Code:
#### `'S'`, `'A'`, `'F'`

In [7]:
compare_rptcode = top_bot_var_compare(df, 'rptcode', rows=10)
compare_rptcode

A                           \
           Sorted Descending %diff  top_avg  bot_avg   
1         trans_cnt_exchg_in  76.0     2.17     0.97   
2          trans_cnt_returns  74.0     4.61     2.13   
3             am_tenure_days  61.0  1858.63   994.64   
4          special_order_amt  57.0  4009.52  2223.92   
5       cp_total_email_trans  55.0   519.98   914.24   
6                multi_value  54.0  4113.66  2374.77   
7             cp_total_trans  53.0   585.92  1013.42   
8   cp_valid_mailing_address  50.0   433.38   718.88   
9            cp_valid_emails  50.0   304.17   505.23   
10               multi_units  49.0    28.66    17.36   

                            F                         \
             Sorted Descending %diff top_avg bot_avg   
1               ft_tenure_days  39.0  357.14  529.83   
2           trans_cnt_exchg_in  35.0    3.49    2.46   
3          trans_cnt_exchg_out  33.0    0.39    0.28   
4        accessory_coats_units  32.0    8.17    5.94   
5    accessory_shoe_care_value  31.0  260.51  190.58   
6              trans_cnt_exchg  29.0    9.47    7.09   
7        accessory_coats_value  29.0  908.62  678.54   
8                  multi_units  29.0   68.75   51.47   
9    accessory_shoe_care_units  29.0   42.50   31.61   
10  accessory_shoe_trees_units  29.0    3.94    2.95   

                            S                           
             Sorted Descending %diff  top_avg  bot_avg  
1              cp_valid_emails  45.0   421.98   667.00  
2               cp_total_trans  45.0   794.57  1252.71  
3         cp_total_email_trans  44.0   673.32  1058.21  
4     cp_valid_mailing_address  44.0   663.65  1041.18  
5                  multi_value  36.0  5450.26  3798.34  
6                  multi_units  35.0    36.19    25.38  
7    accessory_shoe_care_value  32.0   355.25   257.40  
8   accessory_shoe_trees_units  32.0     5.36     3.88  
9          trans_cnt_exchg_out  31.0     0.52     0.38  
10           special_order_amt  31.0  3320.58  2423.55

---
---

# Frequency count of top variables that persist significantly accross aggregations:

### - Class

In [70]:
class_vars_list = list(itertools
                         .chain
                         .from_iterable(compare_class.loc[:, (slice(None), ['Sorted Descending'])].values))

class_vars_freq = sorted(Counter(class_vars_list)
                       .items(), key=lambda kv: kv[1], reverse=True)[:10]

class_vars_freq = [(i[0], str(i[1])+'/7') for i in class_vars_freq]
class_vars_freq.insert(0, ('top variable frequences in class groupings'.upper()))
class_vars_freq

['TOP VARIABLE FREQUENCES IN CLASS GROUPINGS',
 ('multi_units', '6/7'),
 ('multi_value', '5/7'),
 ('trans_cnt_exchg_in', '4/7'),
 ('ft_tenure_days', '4/7'),
 ('cp_total_email_trans', '3/7'),
 ('cp_total_trans', '3/7'),
 ('am_tenure_days', '3/7'),
 ('cp_valid_emails', '3/7'),
 ('cp_valid_mailing_address', '3/7'),
 ('accessory_shoe_trees_units', '3/7')]

### - Volume Band

In [71]:
vband_vars_list = list(itertools
                         .chain
                         .from_iterable(compare_vband.loc[:, (slice(None), ['Sorted Descending'])].values))

vband_vars_freq = sorted(Counter(vband_vars_list)
                       .items(), key=lambda kv: kv[1], reverse=True)[:10]

vband_vars_freq = [(i[0], str(i[1])+'/6') for i in vband_vars_freq]

vband_vars_freq.insert(0, ('top variable frequences in volume_band groupings'.upper()))

### - RPT Code

In [72]:
rptcode_vars_list = list(itertools
                         .chain
                         .from_iterable(compare_rptcode.loc[:, (slice(None), ['Sorted Descending'])].values))

rptcode_vars_freq = sorted(Counter(rptcode_vars_list)
                       .items(), key=lambda kv: kv[1], reverse=True)[:10]

rptcode_vars_freq = [(i[0], str(i[1])+'/3') for i in rptcode_vars_freq]
rptcode_vars_freq.insert(0, ('top variable frequences in rpt_code groupings'.upper()))

 - #### Which variables show up in all three top ten groupings?

In [80]:
freq_list = [class_vars_freq, vband_vars_freq, rptcode_vars_freq]
final_count = ['']

for i in freq_list:
    for k in i[1:]:
        final_count.append(k[0])
                
all_three = [i[0] for i in filter(lambda x: x[1] == 3,Counter(final_count).items())] 
all_three.insert(0, "variables that persist accross top ten AND across all groupings".upper())

---

# Finalize:
## Write results to local machine

 - #### Write `class_vars_freq`, `vband_vars_freq`, `rptcode_vars_freq` to simple .txt file:

In [81]:
freq_list = [class_vars_freq, vband_vars_freq, rptcode_vars_freq, all_three]

txt_destination = './for_presentation/class_vband_rptcode_freq.txt'

header = """[VARIABLE, FREQUENCY COUNT] OF:
\n    CLASSES(7 GROUPINGS),
\n    VOLUME BANDS(6 GROUPINGS), 
\n    RPT CODES(3 GROUPINGS)\n\n"""

def write_freq(freq_list, write_to, header):
    with open(write_to, 'w') as f:
        f.write(header)
        for i in freq_list:
            for k in i:
                f.write(str(k).strip('()')+'\n')   
            f.write('\n')
        
            
            
write_freq(freq_list, txt_destination, header)

 - #### Write DataFrames `compare_class`, `compare_vband`, `compare_rptcode` to xlsx file, each having their own sheet:

In [17]:
df_list = [compare_class, compare_vband, compare_rptcode]
names = ['class_vars', 'vband_vars', 'rptcode_vars']

names_and_dfs = list(zip(names, df_list))

xlsx_destination = './for_presentation/genesco_compare_tables_newest.xlsx'

def excel_writer(df_list, xlsx_destination):
    writer = pd.ExcelWriter(xlsx_destination, 
                            engine='xlsxwriter')
    for i in df_list:
        name = i[0]
        df = i[1]
        df.to_excel(writer, name)
    writer.save()

excel_writer(names_and_dfs, xlsx_destination)